## 1.3 Compute 'regular shape' object orientation using Contour Analysis

In [ ]:
import cv2
import numpy as np
import matplotlib.pyplot as plt

print("OpenCV version: " + cv2.__version__)

### Using Contour Minimum Rectangle Area (`cv2.minAreaRect(contour)`)
- The function `cv2.minAreaRect(contour)` **finds the minimum-area bounding rectangle** that **encloses a given contour**. 
- This rectangle may be **rotated** to fit the shape more efficiently.
    ```python
    rect = cv2.minAreaRect(contour)
    ```
- where `rect` is center `(x,y)`, `(width, height)`, `angle` of rotation<br>
    <img src="resource/boundingrect.png" style="width:200px; margin-top:10px;"></img><br><br>

In [ ]:
# Load and convert the image to grayscale
image = cv2.imread("rotated_blocks.jpg")
gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

# Step 1: Apply simple Binary Thresholding (Inverse Binary Thresholding)
_, binary = cv2.threshold(gray, 70, 255, cv2.THRESH_BINARY)

# Step 2: Apply Canny Edge Detection
edges = cv2.Canny(binary, 50, 200)

# Display the images binary and edges
plt.figure(figsize=(20,10))

plt.subplot(1, 2, 1)
plt.imshow(binary, cmap="gray")
plt.title("Binary Image")
plt.axis("off")

plt.subplot(1, 2, 2)
plt.imshow(edges, cmap="gray")
plt.title("Canny Edge Detection")
plt.axis("off")

plt.show()

- Simple tips choose correct threshold value in `cv2.threshold()`
    - if background is white / brigther than object, use **bigger threshold** and use `cv2.THRESH_BINARY_INV`
    - if background is black / darker than object, use **smaller threshold**  and use `cv2.THRESH_BINARY`

In [ ]:
# Step 3: Find contours
contours, _ = cv2.findContours(binary, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

# Step 4: Calculate minimum area bounding rectangle
output = image.copy()
for contour in contours:
    # Compute minimum area bounding rectangle
    rect = cv2.minAreaRect(contour)
    box = cv2.boxPoints(rect) # Get the 4 corner points of the rectangle
    box = np.intp(box) # Convert the coordinates to integers

    # avoid small rectangles
    if(rect[1][0] < 20 or rect[1][1] < 20):
        continue

    # Draw the rectangle
    cv2.drawContours(output, [box], 0, (0, 0, 255), 1)

    # Get center and angle from rotated rectangle
    center = (int(rect[0][0]), int(rect[0][1]))


    # heading of the angle is always the long side of the rectangle
    angle = (int(rect[2]) - 180) % -180  # Adjust angle to be referenced to the horizontal line
    if rect[1][0] < rect[1][1]:  # If the width is smaller than the height, add -90 degrees
        angle = int(rect[2]) - 90
    angle = abs(angle)


    # Define the length of the orientation lines
    length = 50

    # Black line for original orientation, always referenced as horizontal line
    cv2.line(output, center, (center[0] + length, center[1]), (0, 0, 0), 2)  


    # Compute the end point for the **rotation axis** (red line)
    angle_rad = np.deg2rad(angle)
    end_x = int(center[0] + length * np.cos(angle_rad)) # x = x0 + r * cos(angle)
    end_y = int(center[1] - length * np.sin(angle_rad)) # y = y0 - r * sin(angle)
    cv2.line(output, center, (end_x, end_y), (0, 0, 255), 2)  # Red line for rotation axis


    # Draw angle text near the red line
    text_position = (center[0] - 7, center[1] - 7)
    cv2.putText(output, f"{int(angle)}", text_position, cv2.FONT_HERSHEY_SIMPLEX, 
                0.6, (0, 0, 0), 1, cv2.LINE_AA)

                    
# Show results using Matplotlib
plt.figure(figsize=(20, 6))
plt.imshow(output[:, :, ::-1])  # Convert BGR to RGB
plt.axis("off")
plt.title("Object Orientation with Reference Line")
plt.show()

- Find object orientation using Contour Analysis `cv2.minAreaRect()` has a good result when,
    - the object is blob like object (resular shape & simple).
    - background is solid color (otherwise the detection will be noised by background object).

- When object shape is complex, has concave, irregular shape, branch like,
    - then above method might not perform well.
    <img src="resource/output.png" width="75%">

- ✅ Pros of `cv2.minAreaRect`
    - ✔ Fast and efficient: It works by fitting a rectangle around the object, making it computationally cheaper.
    - ✔ Less sensitive to noise: minAreaRect is more stable against outliers.
    - ✔ Works well for rectangular and box-like objects: It naturally aligns with the object's edges.

- ❌ Cons of `cv2.minAreaRect`
    - ✖ Less accurate for non-rectangular objects: 
        - If the object is not box-shaped (e.g., branch or skeleton like), the orientation may not reflect its true major axis.
    - ✖ Rectangle-based assumption: The computed orientation is tied to the bounding box rather than the object's actual shape.
    - ✖ Angle ambiguity: The returned angle is sometimes inconsistent (e.g., flipping between 0° and 90° for similar shapes).